# Clear Sky Examples

## Overview of Functionality

The captest clear sky functionality is based entirely on wrapping and integrating the clear sky modeling capabilities of the [pvlib-python package](https://pvlib-python.readthedocs.io/en/latest/index.html).  The primary intent of this functionality is to provide the option to easily calculate and plot modeled clear sky data as part of the workflow of loading, visualizing, and validating data from pyranometers.

When setting clear_sky to True when calling the CapData load_data method, the csky function will be called when loading data and the modeled clear sky POA and GHI data will be added to the dataframe.  The plot method will detect these columns and plot them as dashed lines with the measured irradiance data.

Below is a basic example of this functionality using data from the NREL Solar Radiation Research Library.

Andreas, A.; Stoffel, T.; (1981). NREL Solar Radiation Research Laboratory (SRRL): Baseline
Measurement System (BMS); Golden, Colorado (Data); NREL Report No. DA-5500-56488.
http://dx.doi.org/10.5439/1052221 

In [ ]:
import pandas as pd
# import pvlib 
# from pvlib.location import Location
# from pvlib.pvsystem import PVSystem
# from pvlib.tracking import SingleAxisTracker

from bokeh.io import output_notebook
import pytz

output_notebook()

import captest as pvc

In [ ]:
meas_nrel = pvc.CapData()

In [ ]:
loc = {'latitude': 39.742, 'longitude': -105.18, 'altitude': 1828.8, 'tz': 'Etc/GMT+7'}
sys = {'surface_tilt': 40, 'surface_azimuth': 180, 'albedo': 0.2}

In [ ]:
meas_nrel.load_data(fname='nrel_data.csv', source='AlsoEnergy', clear_sky=True, loc=loc, sys=sys)

In [ ]:
meas_nrel.plot(ncols=1, width=800, merge_grps=['irr'], 
               subset=['irr_comb'])

Viewing a few rows of the dataframe within the CapData object shows that the modeled clear sky POA and GHI irradiance have been added as columns when loading the data.

In [ ]:
meas_nrel.df['3/10/2019 12:00':'3/10/2019 12:03']

## Explanation of csky Functions

The clear sky functionality is based around 4 top-level functions:
- pvlib_location
- pvlib_system
- get_tz_index
- csky

### Location and System wrappers: `pvlib_location` and `pvlib_system`

The first two, pvlib_location and pvlib_system, are simply wrappers that generate pvlib location and system objects from a dictionary.  The intent of providing these wrapper functions is to allow the captest user to be able to simply specify a dictionary for each without needing to interact with pvlib directly.  

The location dictionary defined in the example can be used with the `pvlib_location` function to create a pvlib location object.

In [ ]:
loc

In [ ]:
bms_location = pvc.pvlib_location(loc)

In [ ]:
type(bms_location)

In [ ]:
bms_location

Similarly, the `pvlib_system` function can be used to generate a pvlib system object.

Creating a pvlib ModelChain object requires providing a system object with module and inverter parameters defined.  The captest pvlib_system function provides arbitrary module and inverter parameters, so the captest user does not need to find and specify these.  The module and inverter parameters are not used when calculating the clear sky data.

In [ ]:
sys

In [ ]:
bms_system = pvc.pvlib_system(sys)

In [ ]:
type(bms_system)

In [ ]:
bms_system

In [ ]:
bms_system.module_parameters.head()

In [ ]:
bms_system.inverter_parameters.head()

### Get Timezone Index `get_tz_index`

The pvlib methods used to calculate clear sky irradiance require a time-zone aware datetime index as an argument.  The get_tz_index function returns a time-zone aware datetime index given a datetime index or a dataframe.  If the passed argument is already timezone aware the index will be used as is otherwise the timezone will be set using the timezone in the location dictionary.

**Be sure the timezone matches the timezone of the measured data.**

For example, the time stamps of the example data from NREL are in MST and do not adjust for daylight savings, so specifying a timezone that does follow daylight savings, like 'America/Denver' will cause an error. Please refer to the pvlib documentaiton, which has a helpful [section on timezones](https://pvlib-python.readthedocs.io/en/latest/timetimezones.html), for more information.

The example usage of `get_tz_index` below matches the usage within the csky function in the example above. The function returns a time-zone aware datetime index.

In [ ]:
print(meas_nrel.df.index.tz)

In [ ]:
pvc.get_tz_index(meas_nrel.df, loc)

Passing the datetime index rather than the dataframe will return the same time-zone aware index.

In [ ]:
pvc.get_tz_index(meas_nrel.df.index, loc)

The below example shows the behavior if the time_source, dataframe or datetime index, already has a timezone when passed to `get_tz_index`.

In [ ]:
df = meas_nrel.df.copy()

In [ ]:
df.index = df.index.tz_localize('America/Caracas')

In [ ]:
loc

In [ ]:
pvc.get_tz_index(df, loc)

The timezone of the time_source is used and a warning is raised to alert the user that the timezone of the time_source and the passed location dictionary do not match.

### Clear Sky Function `csky`

The clear sky function calls `pvlib_location`, `pvlib_system`, and `get_tz_index` and genarates pvlib objects to calculate modeled clear sky GHI and POA.  The essential functionality is shown in the example where the clear sky POA and GHI are added to the imported data.  `csky` can also return any componenet of the modeled csky data directly as shown below.

In [ ]:
poa_ghi = pvc.csky(meas_nrel.df, loc=loc, sys=sys, concat=False, output='both')
poa_ghi['3/10/2019 12:00':'3/10/2019 12:05']

In [ ]:
all_irrad_comp = pvc.csky(meas_nrel.df, loc=loc, sys=sys, concat=False, output='all')
all_irrad_comp['3/10/2019 12:00':'3/10/2019 12:05']

## Tracking System Example

In [ ]:
loc = {'latitude': 43.18416667, 'longitude': -75.4327778, 'altitude': 496, 'tz': 'America/New_York'}
sys = {'surface_tilt': 25, 'surface_azimuth': 180, 'albedo': 0.2}

# or for tracker

#sys = {'axis_tilt': 0, 'axis_azimuth': 0, 'max_angle': 90, 'backtrack': True, 'gcr': 0.2, 'albedo': 0.2}

In [ ]:
mm = pvc.CapData()

In [ ]:
mm.load_data(path='./', fname='martino_test_data.csv', clear_sky=True, loc=loc, sys=sys, source='AlsoEnergy')

In [ ]:
# mm.df.head()

In [ ]:
mm.plot(subset=['irr_comb'], width=800)

In [ ]:
pwd

In [ ]:
nrel = pd.read_csv('./nrel_data.csv', parse_dates=[['DATE (MM/DD/YYYY)', 'MST']])

In [ ]:
nrel.set_index('DATE (MM/DD/YYYY)_MST', inplace=True)

In [ ]:
nrel.head()

In [ ]:
nrel.rename(columns={'Global 40-South CMP11 [W/m^2]': 'POA 40-South CMP11 [W/m^2]'}, inplace=True)

In [ ]:
nrel.to_csv('nrel_data_datetime_index.csv')

In [ ]:
loc_obj = pvc.pvlib_location(loc)
sys_obj = pvc.pvlib_system(sys)

In [ ]:
mc = pvlib.modelchain.ModelChain(sys_obj, loc_obj)

In [ ]:
mc.complete_irradiance(meas_nrel.df, )

In [ ]:
mc.prepare_inputs(meas_nrel.df, )

In [ ]:
loc = {'latitude': 30.274583,
            'longitude': -97.740352,
            'altitude': 500,
            'tz': 'America/Chicago'}

sys = {'surface_tilt': 20,
            'surface_azimuth': 180,
            'albedo': 0.2}

# or for tracker

#sys = {'axis_tilt': 0, 'axis_azimuth': 0, 'max_angle': 90, 'backtrack': True, 'gcr': 0.2, 'albedo': 0.2}

In [ ]:
meas = pvc.CapData()
df = meas.load_das('../tests/data/', 'example_meas_data.csv')

In [ ]:
print(df.index.tz)

In [ ]:
csky_ghi_poa = pvc.csky(df, loc=loc, sys=sys)

In [ ]:
print(csky_ghi_poa.index.tz)

In [ ]:
meas.plot(subset=['irr_comb'], width=800)

The test data is modified tmy3 data.  TMY3 data is in standard time for the entire year (no DST).  This test data period is in October during DST, so the timezone specified for the location must be central time w/o DST, which is UTC-6.  This results in data aligment, but not perfect.  I think the remaining lack of alignment is due to modifications to the data.

Need to develop a few tests for data aligment that use validated test data.
- very clear days
- known timezone and no uncert or issues with timestamps

In [ ]:
csky_cd = pvc.CapData()

In [ ]:
csky_cd.df = pvc.csky(meas.df, loc=loc, sys=sys, concat=False, output='both')

In [ ]:
csky_cd._CapData__set_trans()

In [ ]:
csky_cd.df.index = csky_cd.df.index.tz_localize(None)

In [ ]:
csky_cd.plot(subset=['irr_comb'], width=800)

In [ ]:
pvlib.location.Location()

In [ ]:
pvc.capdata.csky_ghi()

In [ ]:
meas.load_das()

In [ ]:
type(pd.Series())

In [ ]:
cd = pvc.CapData()

I'm thinking the signature for the load_data method would change to something like the below where the loc and sys keywords are dictionaries of the data required to instantiate pvlib Location and System objects.
- clear_sky default would be False with loc=None and sys=None
- if clear_sky is true would require these dictionaries

In [ ]:
cd.load_data(path='./data/', fname=None, set_trans=True, source=None, load_pvsyst=False,
             clear_sky=True, loc=loc, sys=sys, **kwargs)
"""

clear_sky : bool or str, default False
    Default is to not calculate clear sky irradiance when loading data.  Set to true to
    use the pvlib package to calculate clear sky poa and ghi.  When True, requires passing
    a dictionary to the loc and sys keywords as defined below.
    
    loc = {'latitude': float,
           'longitude': float,
           'altitude': float/int,
           'tz': }
    sys = {}
    
    Set to 'ghi' to calculate only global horizontal clear sky.  This option only requires
    a dictionary passed to the 'loc' keyword.
"""

Where to handle the timezone issue?  Require dataframe passed to have timezone?

In [ ]:
def csky_ghi(df, loc):
    """
    Creates a pvlib location object and returns clear sky GHI irradiance using the get_clearsky method.
    
    Parameters
    ----------
    df : dataframe with datetime index
        Clear sky ghi is calculated for times in dataframe index.
    loc : dict
        Dictionary of values required to instantiate a pvlib Location object.
    
    Returns
    -------
    Tuple of the location object and the clear sky GHI as a series.
    """
    location = pvlib.location.Location(**loc_dict)
    return (location, location.get_clearsky(times=ix or df.index))   

Example dictionaries:

In [ ]:
loc = {'latitude': 30.274583, 'longitude': -97.740352, 'altitude': 500, 'tz': 'America/Chicago'}
sys = {'surface_tilt': 20, 'surface_azimuth': 180, 'albedo': 0.2}

# or for tracker

#sys = {'axis_tilt': 0, 'axis_azimuth': 0, 'max_angle': 90, 'backtrack': True, 'gcr': 0.2, 'albedo': 0.2}

Use something like the below to check if user passes a system dictionary for a tracker or fixed tilt and create the appropriate pvlib object.

In [ ]:
trck_kwords = ['axis_tilt', 'axis_azimuth', 'max_angle', 'backtrack', 'gcr']
if any(kword in sys.keys() for kword in trck_kwords):
    system = pvlib.tracking.SingleAxisTracker(**sys, module_parameters=sandia_module, inverter_parameters=cec_inverter)
else:
    system = pvlib.pvsystem.PVSystem(**sys, module_parameters=sandia_module, inverter_parameters=cec_inverter)

I was mostly testign the keyword unpacking in the below fucntions; I don't think there necessarily needs to be functions for this step.

In [ ]:
pvlib.tracking.SingleAxisTracker()

In [ ]:
def make_loc(loc_dict):
    return pvlib.location.Location(**loc_dict)

Instatiating a model chain object requires a system object that has modules and inverters specified.  This is kind of annoying for this use case as I'm not going past the step of calculating the total poa irradiance.  I am thinking for now that I will just define a random module and inverter within captest to work around this and possibly submit a request to pvlib to change this behavior.  Then if it changes, I can just remove the module and inverter definitions.

In [ ]:
def make_sys(sys_dict):
    sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
    cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
    sandia_module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
    cec_inverter = cec_inverters['ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_']
    system = pvlib.pvsystem.PVSystem(**sys_dict, module_parameters=sandia_module, inverter_parameters=cec_inverter)
    return system

In [ ]:
location = make_loc(loc)

In [ ]:
system = make_sys(sys)

In [ ]:
# location = pvlib.location.Location(latitude=30.2, longitude=-60)

In [ ]:
# sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
# cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')
# sandia_module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
# cec_inverter = cec_inverters['ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_']
# system = pvlib.pvsystem.PVSystem(surface_tilt=20, albedo=0.2, module_parameters=sandia_module, inverter_parameters=cec_inverter)
# track_sys = pvlib.tracking.SingleAxisTracker(module_parameters=sandia_module, inverter_parameters=cec_inverter, albedo=0.2)

In [ ]:
mc = pvlib.modelchain.ModelChain(system, location)

In [ ]:
ix = pd.DatetimeIndex(start='1/1/2019', periods=8760, freq='H', tz='America/Chicago')

In [ ]:
ix.tz_localize(None)

In [ ]:
ix_no_tz[1631:1640]

In [ ]:
ix_no_tz[7343:7354]

In [ ]:
ix.tz_localize(None)

In [ ]:
import pytz

In [ ]:
pytz.timezone('America/Chicago') == ix.tz

In [ ]:
mc.prepare_inputs(times=ix)

In [ ]:
cols = ['poa_global', 'poa_direct', 'poa_diffuse', 'poa_sky_diffuse', 'poa_ground_diffuse', 'poa_ground_diffuse']
all(col in mc.total_irrad.columns for col in cols)

In [ ]:
pd.concat([mc.total_irrad, ghi], axis=1)

In [ ]:
pd.DataFrame({'poa_mod_csky': mc.total_irrad['poa_global'], 'ghi_mod_csky': ghi['ghi']})

In [ ]:
ghi = location.get_clearsky(times=ix)

In [ ]:
ghi

In [ ]:
mc.total_irrad.loc['2019-06-01':'2019-06-05', 'poa_global'].plot()

In [ ]:
pwd

In [ ]:
cd = pvc.CapData()
df = meas.load_das('../tests/data/', 'example_meas_data.csv')

In [ ]:
meas.df.index

In [ ]:
pvc.get_tz_index(meas.df.index, loc).tz_localize(None, loc['tz'])

In [ ]:
tst = pvc.CapData()

In [ ]:
tst_df = tst.load_das('../tests/data/', 'example_meas_data.csv')

In [ ]:
pvc.get_tz_index(tst_df, loc)

In [ ]:
meas.df.head()

In [ ]:
meas.trans

In [ ]:
meas.plot(width=1000, ncols=1)

In [ ]:
pvc.csky(df, loc=loc, sys=sys).loc[:, ['met1 poa_refcell', 'poa_mod_csky']].plot()

In [ ]:
dict

In [ ]:
pvc.csky(df, loc=loc, sys=sys, concat=False, output='all')

In [ ]:
'ghi' in df.columns

In [ ]:
df.index

In [ ]:
1440/(24 * (60/5))

In [ ]:
ix_3days = pd.DatetimeIndex(start='11/3/2018', periods=864, freq='5min', tz='America/Chicago')

In [ ]:
12*24*2

In [ ]:
ix_3days

In [ ]:
ix_2days = pd.DatetimeIndex(start='3/9/2019', periods=576, freq='5min', tz='America/Chicago')

In [ ]:
ix_dst = ix_3days.append(ix_2days)

In [ ]:
df.index = ix_dst

In [ ]:
df.columns

In [ ]:
df['2018-11-04'].index[12:40]

In [ ]:
df['2019-03-10'].index[12:40]